### 1. Load library, function, and other inputs ###

In [1]:
# This script is used to compare ensemble outputs with NLDAS data
import os
os.environ["PROJ_LIB"] = '/glade/u/home/hongli/tools/miniconda3/envs/conda_hongli/share/proj'

from mpl_toolkits.basemap import Basemap
from mpl_toolkits.axes_grid1 import make_axes_locatable
from pyproj import Proj
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
import xarray as xr
import datetime
import random
from tqdm import tqdm

def read_nldas(file_dir, date_start_obj, date_end_obj, mask_xy):
    start_yr = int(datetime.datetime.strftime(date_start_obj,'%Y'))
    end_yr = int(datetime.datetime.strftime(date_end_obj,'%Y'))
    
    for yr in range(start_yr, end_yr+1):
        nldas_file = 'NLDAS_'+str(yr)+'.nc'
        nldas_path = os.path.join(file_dir, nldas_file)
        f_nldas = xr.open_dataset(nldas_path)
        
        if yr == start_yr:
            pcp_concat = f_nldas['pcp'].values[:] # (time, y, x). unit: mm/day
            tmean_concat = f_nldas['t_mean'].values[:] # (time, y, x). unit: degC
#             t_range_concat = f_nldas['t_range'].values[:]
            time_concat = f_nldas['time'].values[:]
        else:
            pcp_concat = np.concatenate((pcp_concat, f_nldas['pcp'].values[:]), axis = 0)
            tmean_concat = np.concatenate((tmean_concat, f_nldas['t_mean'].values[:]), axis = 0)
#             trange_concat = np.concatenate((trange_concat, f_nldas['t_range'].values[:]), axis = 0)
            time_concat = np.concatenate((time_concat, f_nldas['time'].values[:]), axis = 0)

    # get time mask from nldas data
    time_obj = pd.to_datetime(time_concat)    
    
    mask_t  = (time_obj >= date_start_obj) & (time_obj <= date_end_obj) 
    time_obj = time_obj[mask_t]
    pcp_concat=np.where(mask_xy==0,np.nan,pcp_concat[mask_t,:,:])  # convert masked values to nan  
    tmean_concat=np.where(mask_xy==0,np.nan,tmean_concat[mask_t,:,:])  
    return time_obj, pcp_concat, tmean_concat#, t_range_concat

def read_ens_mb(file_dir, mb, date_start_obj, date_end_obj, mask_xy):
    start_yr = int(datetime.datetime.strftime(date_start_obj,'%Y'))
    end_yr = int(datetime.datetime.strftime(date_end_obj,'%Y'))
    
    for yr in range(start_yr, end_yr+1):                
        filename='ens_forc.%d.%03d.nc'%(yr,mb)
        file = os.path.join(file_dir, filename)
        f=xr.open_dataset(file)
        time = f['time'].values[:]
        pcp = f['pcp'].values[:]
        tmean = f['t_mean'].values[:]
#        trange = f['t_range'].values[:]
        
        if yr == start_yr:
            time_concat = time
            pcp_concat = pcp
            tmean_concat = tmean
#             trange_concat = trange
        else:
            time_concat = np.concatenate((time_concat,time), axis=0) # (time)
            pcp_concat = np.concatenate((pcp_concat, pcp), axis=0) # (time,y,x)
            tmean_concat = np.concatenate((tmean_concat, tmean), axis=0)
#             trange_concat = np.concatenate((trange_concat, trange), axis=0)            
    
    time_obj = pd.to_datetime(time_concat)   
    
    mask_t  = (time_obj >= date_start_obj) & (time_obj <= date_end_obj) 
    time_obj = time_obj[mask_t]
    pcp_concat=np.where(mask_xy==0,np.nan,pcp_concat[mask_t,:,:])  # convert masked values to nan  
    tmean_concat=np.where(mask_xy==0,np.nan,tmean_concat[mask_t,:,:])  
    return time_obj, pcp_concat, tmean_concat#, trange_concat

def read_ens_std(file_dir, date_start_obj, date_end_obj, mask_xy):
    start_yr = int(datetime.datetime.strftime(date_start_obj,'%Y'))
    end_yr = int(datetime.datetime.strftime(date_end_obj,'%Y'))
    
    for yr in range(start_yr, end_yr+1):                
        
        filename='ens_forc.%d.ensstd.nc'%(yr)
        file = os.path.join(file_dir, filename)
        f=xr.open_dataset(file)
        time = f['time'].values[:]
        pcp_error = f['pcp'].values[:] # std
        tmean_error = f['t_mean'].values[:]
#         trange_error = f['t_range'].values[:]
        
        if yr == start_yr:
            time_concat = time
            pcp_error_concat = pcp_error
            tmean_error_concat = tmean_error
#             trange_error_concat = trange_error
        else:
            time_concat = np.concatenate((time_concat,time), axis=0) # (time)
            pcp_error_concat = np.concatenate((pcp_error_concat, pcp_error), axis=0) # (time,y,x)
            tmean_error_concat = np.concatenate((tmean_error_concat, tmean_error), axis=0)
#             trange_error_concat = np.concatenate((trange_error_concat, trange_error), axis=0)
            
    time_obj = pd.to_datetime(time_concat)        
    mask_t  = (time_obj >= date_start_obj) & (time_obj <= date_end_obj) 
    time_obj = time_obj[mask_t]
    pcp_error_concat=np.where(mask_xy==0,np.nan,pcp_error_concat[mask_t,:,:])  # convert masked values to nan  
    tmean_error_concat=np.where(mask_xy==0,np.nan,tmean_error_concat[mask_t,:,:])  
    return time_obj, pcp_error_concat, tmean_error_concat#, trange_error_concat

def plot_basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,ax,lat_0,lon_0,ny,nx):

    m = Basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,resolution='l',projection='cyl', ax=ax)   
#     m = Basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,resolution='l',projection='tmerc', ax=ax,lat_0=lat_0,lon_0=lon_0)

    m.drawstates(linewidth=0.5, linestyle='solid', color='grey')
    m.drawcountries(linewidth=0.5, linestyle='solid', color='k')
    m.drawcoastlines(linewidth=.25, linestyle='solid', color='k')
    return m

def set_box_color(bp, color):
    plt.setp(bp['boxes'], color=color)
    plt.setp(bp['whiskers'], color=color)
    plt.setp(bp['caps'], color=color)
    plt.setp(bp['medians'], color=color)
    return

# set the colormap and centre the colorbar
class MidpointNormalize(mpl.colors.Normalize):
    """Normalise the colorbar.
    source: http://chris35wills.github.io/matplotlib_diverging_colorbar/
    e.g. im=ax1.imshow(array, norm=MidpointNormalize(midpoint=0.,vmin=-300, vmax=1000))    
    """
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        mpl.colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y), np.isnan(value))

#======================================================================================================
# main script
root_dir = '/glade/u/home/hongli/scratch/2020_04_21nldas_gmet'   
nldas_dir = os.path.join(root_dir,'data/nldas_daily_utc_convert')
gridinfo_file = os.path.join(root_dir,'data/nldas_topo/conus_ens_grid_eighth.nc')

result_dir = os.path.join(root_dir,'test_uniform_perturb')
test_folders = [d for d in os.listdir(result_dir)]
test_folders = sorted(test_folders)
test_folder =test_folders[-2]

regress_subforlder = 'gmet_regr'
ens_subforlder = 'gmet_ens_bc'
ens_stat_subforlder='gmet_ens_bc_summary'

ens_num = 100 #100
time_format = '%Y-%m-%d'

dpi_value = 150
date_starts = ['2016-02-01','2016-08-01'] # list of start dates of plot time periods
date_ends = ['2016-02-29','2016-08-31']   # list of end dates of plot time periods
grids = [[40,265],[40,265]]               # xid and yid of the spatial distribution
# date_starts = ['2016-08-09']
# date_ends = ['2016-08-20']

random.seed(10)

output_dir=os.path.join(root_dir, 'scripts/step35_statistical_perturb_NLDAS_v2')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print('Done')

Done


### 1. Read data ###

In [2]:
#======================================================================================================
# PART 1. Read gridinfo mask
print('Read gridinfo mask')
# get xy mask from gridinfo.nc
f_gridinfo = xr.open_dataset(gridinfo_file)
mask_xy = f_gridinfo['mask'].values[:] # (y, x). 1 is valid. 0 is invalid.
latitude = f_gridinfo['latitude'].values[:]
longitude = f_gridinfo['longitude'].values[:]

# prepare for basemap plot
llcrnrlon = longitude[0,0]
urcrnrlon = longitude[-1,-1]
llcrnrlat = latitude[0,0]
urcrnrlat = latitude[-1,-1]
lat_0=0.5*(llcrnrlat+urcrnrlat)
lon_0=0.5*(llcrnrlon+urcrnrlon)
(ny,nx)=np.shape(longitude)

# PART 2. read and save data for different time periods (pt)
data_dic={}
for pt in range(len(date_starts)):
# for pt in range(1):
    key=str(pt)
    data_dic[key]={}
    
    #======================================================================================================
    # PART 2.1 determine useful time period
    date_start_obj = datetime.datetime.strptime(date_starts[pt], time_format)
    date_end_obj = datetime.datetime.strptime(date_ends[pt], time_format)
    [xid,yid]=grids[pt]
    lons = longitude[xid,yid]
    lats = latitude[xid,yid]   
    print('---- %s - %s ----'%(date_starts[pt],date_ends[pt]))
    
    #======================================================================================================
    # PART 2.2 read 
    print('Read nldas data')
    time_nldas, pcp_nldas, tmean_nldas = read_nldas(nldas_dir,date_start_obj, date_end_obj, mask_xy)

    print('Read ensemble std')
    file_dir = os.path.join(result_dir,test_folder,ens_stat_subforlder)
    time_std, pcp_gmet_std, tmean_gmet_std = read_ens_std(file_dir, date_start_obj, date_end_obj, mask_xy)   

    print('Read ensemble data')
    file_dir = os.path.join(result_dir,test_folder,ens_subforlder)

    pbar = tqdm(total=ens_num)
    for mb in range(ens_num):
        time_mb, pcp_mb, tmean_mb = read_ens_mb(file_dir, mb+1, date_start_obj, date_end_obj, mask_xy)
        (nt,nx,ny) = np.shape(pcp_mb) 
        pcp_mb,tmean_mb = np.reshape(pcp_mb,(nt,nx,ny,1)),np.reshape(tmean_mb,(nt,nx,ny,1))

        if mb == 0:
            time_gmet_ens,pcp_gmet_ens,tmean_gmet_ens = time_mb,pcp_mb,tmean_mb
        else:
            pcp_gmet_ens = np.concatenate((pcp_gmet_ens,pcp_mb), axis=-1)
            tmean_gmet_ens = np.concatenate((tmean_gmet_ens,tmean_mb), axis=-1)
        pbar.update(1)
    pbar.close()
    del time_mb, pcp_mb, tmean_mb
    
    #=====================================================
    # PART 2.3 perturb NLADS precipitation 
    print('perturb NLADS Precip')
    # --- assume Precip ~ Gamma ---
    (nt,nx,ny)=np.shape(pcp_nldas)
    pcp_stat_ens = np.empty((nt,nx,ny,ens_num))
    pcp_stat_ens[:] = np.NaN

    cp2 = 0.3**2 #0.25**2
    shape = 1/cp2 * np.ones_like(pcp_nldas)
    scale = cp2 * pcp_nldas

    pbar = tqdm(total=ens_num)
    for mb in range(ens_num):
        pcp_stat_ens[...,mb]=np.random.gamma(shape, scale)
        pbar.update(1)
    pbar.close()

    #======================================================================================================
    # PART 2.4  perturb NLADS Tmean
    print('perturb NLADS Tmean')
    # ---assume mean Temp ~ Normal ---
    (nt,nx,ny)=np.shape(tmean_nldas)
    tmean_stat_ens = np.empty((nt,nx,ny,ens_num))
    tmean_stat_ens[:] = np.NaN

    mean = tmean_nldas
    std = 0.29 # degree

    pbar = tqdm(total=ens_num)
    for mb in range(ens_num):
        tmean_stat_ens[...,mb]=np.random.normal(mean, std)
        pbar.update(1)
    pbar.close()
    
    #======================================================================================================
    # PART 2.3.6 save data to dic
    data_dic[key] = {'time_nldas':time_nldas, 
                     'pcp_nldas':pcp_nldas, 'tmean_nldas':tmean_nldas,
                     'pcp_stat_ens':pcp_stat_ens, 'tmean_stat_ens':tmean_stat_ens,
                     'pcp_gmet_ens':pcp_gmet_ens, 'tmean_gmet_ens':tmean_gmet_ens, 
                     'pcp_gmet_std':pcp_gmet_std, 'tmean_gmet_std':tmean_gmet_std}

print('Done')

Read gridinfo mask
---- 2016-02-01 - 2016-02-29 ----
Read nldas data
Read ensemble std


  0%|          | 0/100 [00:00<?, ?it/s]

Read ensemble data


100%|██████████| 100/100 [04:02<00:00,  2.42s/it]


perturb NLADS Precip


...

perturb NLADS Precip


100%|██████████| 100/100 [00:33<00:00,  2.98it/s]


perturb NLADS Tmean


100%|██████████| 100/100 [00:21<00:00,  4.70it/s]

Done


### 2. Plot ###

In [3]:
def create_colormap(cmap):
    # create a white-cmap linear colormap
    # reference: https://stackoverflow.com/questions/25408393/getting-individual-colors-from-a-color-map-in-matplotlib
    cmap = mpl.cm.get_cmap(cmap) #‘viridis’，'jet' # get the blue color of jet 
    c0 = cmap(0.0)
    top = mpl.colors.LinearSegmentedColormap.from_list("", ["white",c0])

    # combine two liner colormaps to create a
    # reference: https://matplotlib.org/3.1.0/tutorials/colors/colormap-manipulation.html
    bottom = mpl.cm.get_cmap(cmap) #'jet','viridis'
#     newcolors = np.vstack((top(np.linspace(0, 1, int(256*0.01))),bottom(np.linspace(0, 1, int(256*0.99)))))
    newcolors = bottom(np.linspace(0.3, 1, int(256)))
    newcmp = mpl.colors.LinearSegmentedColormap.from_list("NewCmap", newcolors)
    return newcmp

#======================================================================================================    
# create a white-jet linear colormap
print('Create colormap')
# reference: https://stackoverflow.com/questions/25408393/getting-individual-colors-from-a-color-map-in-matplotlib
cmap = mpl.cm.get_cmap('jet') # get the blue color of jet 
c0 = cmap(0.1)
# top = mpl.colors.LinearSegmentedColormap.from_list("", ["white",c0])
top = mpl.colors.LinearSegmentedColormap.from_list("", ["white","white"])

# combine two liner colormaps to create a
# reference: https://matplotlib.org/3.1.0/tutorials/colors/colormap-manipulation.html
bottom = plt.cm.bwr
newcolors = bottom(np.linspace(0.1, 1, 256))
NewBwr = mpl.colors.LinearSegmentedColormap.from_list("NewBwr", newcolors)

#======================================================================================================    
# PART 3. plot
# --- plot precipitation in space
nrow = len(date_starts) # Wet, dry events
ncol_pcp_space = 3 # NLDAS, GMET std, statistical std    
fig_pcp_space = plt.figure(figsize=(7.08,7.08*0.4), constrained_layout=True)
gs_pcp_space = fig_pcp_space.add_gridspec(nrow, ncol_pcp_space,height_ratios=[1, 1.05])   

# --- plot precipitation in time
nrow = len(date_starts) # Wet, dry events
ncol_pcp_time = 1 # time series    
fig_pcp_time = plt.figure(figsize=(7.08,7.08*0.4), constrained_layout=True)
gs_pcp_time = fig_pcp_time.add_gridspec(nrow, ncol_pcp_time)   

# --- plot Tmean in space
nrow = len(date_starts) # Wet, dry events
ncol_tmean_space = 2 # NLDAS, GMET std    
fig_tmean_space = plt.figure(figsize=(7.08*2/3.0,7.08*0.4), constrained_layout=True)
gs_tmean_space = fig_tmean_space.add_gridspec(nrow, ncol_tmean_space,height_ratios=[1, 1.05])   

# --- plot Tmean in time
nrow = len(date_starts) # Wet, dry events
ncol_tmean_time = 1 # time series    
fig_tmean_time = plt.figure(figsize=(7.08,7.08*0.4), constrained_layout=True)
gs_tmean_time = fig_tmean_time.add_gridspec(nrow, ncol_tmean_time)   

for pt in range(nrow):
# for pt in range(1):
    key=str(pt)    

    # PART 3.1 extract data
    date_start_obj = datetime.datetime.strptime(date_starts[pt], time_format)
    date_end_obj = datetime.datetime.strptime(date_ends[pt], time_format)
    print('---- %s - %s ----'%(date_starts[pt],date_ends[pt]))

    time_nldas = data_dic[key]['time_nldas']
    
    pcp_nldas = data_dic[key]['pcp_nldas']       #(t,y,x)
    pcp_stat_ens = data_dic[key]['pcp_stat_ens'] #(t,y,x,mb)
    pcp_gmet_ens = data_dic[key]['pcp_gmet_ens'] #(t,y,x,mb)
    pcp_gmet_std = data_dic[key]['pcp_gmet_std'] #(t,y,x)
    
    tmean_nldas = data_dic[key]['tmean_nldas']
    tmean_stat_ens = data_dic[key]['tmean_stat_ens']
    tmean_gmet_ens = data_dic[key]['tmean_gmet_ens']
    tmean_gmet_std = data_dic[key]['tmean_gmet_std']
    
    #=====================================================
    # PART 3.2 calculate Precip mean, std, vmin and vmax for plot
    pcp_nldas_mean = np.nanmean(pcp_nldas, axis=0) #(y,x)
    pcp_stat_std_mean = np.nanmean(np.nanstd(pcp_stat_ens, axis=-1), axis=0) 
#     pcp_gmet_std_mean = np.nanmean(np.nanstd(pcp_gmet_ens, axis=-1), axis=0)
    pcp_gmet_std_mean = np.nanmean(pcp_gmet_std, axis=0)
                      
#     vmin_pcp = np.nanmin(pcp_nldas_mean)
#     vmax_pcp = np.nanpercentile(pcp_nldas_mean,99)    
#     vmin_pcp_std = np.nanmin([np.nanmin(pcp_stat_std_mean),np.nanmin(pcp_gmet_std_mean)])
#     vmax_pcp_std = np.nanmax([np.nanpercentile(pcp_stat_std_mean,99.5), np.nanpercentile(pcp_gmet_std_mean,99.5)])
    vmin_pcp,vmax_pcp = 0,9.5   
    vmin_pcp_std,vmax_pcp_std = 0,4.8

    #=====================================================
    # PART 3.3 calculate Tmean mean, std, vmin and vmax for plot
    tmean_nldas_mean = np.nanmean(tmean_nldas, axis=0) #(y,x)
    tmean_stat_std_mean = np.nanmean(np.nanstd(tmean_stat_ens, axis=-1), axis=0)                      
#     tmean_gmet_std_mean = np.nanmean(np.nanstd(tmean_gmet_ens, axis=-1), axis=0)
    tmean_gmet_std_mean = np.nanmean(tmean_gmet_std, axis=0)

#     vmin_tmean = np.nanmin(tmean_nldas_mean)
#     vmax_tmean = np.nanmax(tmean_nldas_mean)    
#     vmin_tmean_std = np.nanmin([np.nanmin(tmean_stat_std_mean),np.nanmin(tmean_gmet_std_mean)])
#     vmax_tmean_std = np.nanmax([np.nanpercentile(tmean_stat_std_mean,99.5),np.nanpercentile(tmean_gmet_std_mean,99.5)])    
    vmin_tmean,vmax_tmean = 0,38    
    vmin_tmean_std, vmax_tmean_std = 0, 1.7

    #=====================================================
    # PART 3.4 plot Precip in space
    # reference: https://stackoverflow.com/questions/16592222/matplotlib-group-boxplots    
    print('plot Precip in space')
    for j in range(ncol_pcp_space):
        if j==0:
            newcmp=create_colormap(plt.cm.terrain_r)
            cmap,vmin,vmax=newcmp, vmin_pcp, vmax_pcp #plt.cm.Blues #plt.cm.terrain_r
            data,title_str=pcp_nldas_mean,'(a) NLDAS Precip'               
        else:
            cmap,vmin,vmax=plt.cm.Blues, vmin_pcp_std, vmax_pcp_std #plt.cm.jet_r
            if j == 1:
                data,title_str=pcp_gmet_std_mean,'(b) Spatial regression Std Dev'
            elif j==2:
                data,title_str=pcp_stat_std_mean,'(c) Parametric Std Dev'

        # plot Basemap
        ax = fig_pcp_space.add_subplot(gs_pcp_space[pt,j])
        m = plot_basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,ax,lat_0,lon_0,ny,nx) # plot Basemap 

        # plot data
        im = m.pcolormesh(longitude,latitude,data,shading='flat',latlon=True,cmap=cmap,vmin=vmin,vmax=vmax)

        # set colorbar
        if pt!=0:
            cbar = fig_pcp_space.colorbar(im,ax=ax,pad=0.002,orientation="horizontal")  
            cbar.ax.tick_params(labelsize='xx-small',pad=0.01, length=2)             
            cbar.set_label(label='(mm/day)', size='xx-small', rotation='horizontal', labelpad=-0.5)

        # set title
        if pt == 0:
            ax.set_title(title_str, fontsize='xx-small', fontweight='semibold')

        # set ylabel
        if j == 0:
            ax.set_ylabel(datetime.datetime.strftime(time_nldas[0],'%B %Y'), fontsize='xx-small', fontweight='semibold')

        # plot grids location
        if j == 0:
            m.scatter(lons, lats, marker = 'o', color='none', 
                      edgecolors='red', zorder=5, alpha=0.8)#magenta
                
    #=====================================================
    # PART 3.5 plot Precip in time
    print('plot Precip in time')
    ax = fig_pcp_time.add_subplot(gs_pcp_time[pt,:])

    # extract point data
    nldas_point = pcp_nldas[:,xid,yid]            
    data_left = pcp_gmet_ens[:,xid,yid,:]
    data_right = pcp_stat_ens[:,xid,yid,:]  

    data_left = data_left.tolist()
    data_right = data_right.tolist()

    # boxplot left data
    positions=np.array(range(len(data_left)))*2.0-0.4
    color='blue'
    flierprops = dict(marker='+', markerfacecolor='none', markersize=2,
                  linestyle='none', markeredgecolor=color)
    bpl = ax.boxplot(data_left, positions=positions, widths=0.6, 
                     flierprops=flierprops, medianprops=dict(alpha=.8))
    set_box_color(bpl, color) # colors are from http://colorbrewer2.org/
    sc = ax.scatter(positions,nldas_point,s=4, c='k',alpha=1,zorder=2)

    # boxplot right data
    positions=np.array(range(len(data_left)))*2.0+0.4
    color='red'
    flierprops = dict(marker='+', markerfacecolor='none', markersize=2,
                  linestyle='none', markeredgecolor=color)
    bpr = ax.boxplot(data_right, positions=positions, widths=0.6, 
                     flierprops=flierprops, medianprops=dict(alpha=.8))
    set_box_color(bpr, color)
    sc = ax.scatter(positions,nldas_point,s=4, c='k',alpha=1,zorder=2)

    # Add a horizontal grid to the plot, but make it very light in color
    # so we can use it for reading data values but not be distracting
    ax.yaxis.grid(True, linestyle='-', which='major', color='lightgrey',alpha=0.5)
    ax.set_axisbelow(True)

    # xticks
    ticks = [datetime.datetime.strftime(t,'%d') for t in time_nldas]
    ax.set_xticklabels(ticks)
    ax.set_xticks(np.array(range(len(data_left)))*2.0)
    ax.tick_params(axis='both',labelsize='xx-small')

    # draw temporary red and blue lines and use them to create a legend
    ax.legend([sc, bpl["boxes"][0], bpr["boxes"][0]], ['NLADS', 'Spatial regression', 'Parametric'], 
              loc='upper right',fontsize='xx-small')

    ax.set_xlabel('Date', fontsize='xx-small') 
    ax.set_ylabel('Precip (mm/day)', fontsize='xx-small') 

    # set title
    title_str= '(%s) Precip in %s'%(chr(ord('a')+pt), datetime.datetime.strftime(time_nldas[0],'%B %Y'))                         
    ax.set_title(title_str, fontsize='xx-small', fontweight='semibold')    
        
    #=====================================================
    # PART 3.6 plot Tmean in space
    # reference: https://stackoverflow.com/questions/16592222/matplotlib-group-boxplots
    print('plot Tmean in space')
    for j in range(ncol_tmean_space):
        if j==0:
            cmap,vmin,vmax=plt.cm.jet, vmin_tmean, vmax_tmean #plt.cm.Blues
            data,title_str=tmean_nldas_mean,'(a) NLDAS Tmean'               
        else:
            cmap,vmin,vmax=NewBwr, vmin_tmean_std, vmax_tmean_std #plt.cm.bwr
            if j == 1:
                data,title_str=tmean_gmet_std_mean,'(b) Spatial regression Std Dev'
            elif j==2:
                data,title_str=tmean_stat_std_mean,'(c) Parametric Std Dev'

        # plot Basemap
        ax = fig_tmean_space.add_subplot(gs_tmean_space[pt,j])
        m = plot_basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,ax,lat_0,lon_0,ny,nx) # plot Basemap 

        # plot data
        im = m.pcolormesh(longitude,latitude,data,shading='flat',latlon=True,cmap=cmap,vmin=vmin,vmax=vmax)
        
        if j == 0:
            im = m.pcolormesh(longitude,latitude,data,shading='flat',latlon=True,cmap=cmap,vmin=vmin,vmax=vmax)
        elif j>=1:
            im = m.pcolormesh(longitude,latitude,data,shading='flat',latlon=True,cmap=cmap,vmin=vmin,vmax=vmax,
                             norm=MidpointNormalize(midpoint=std,vmin=vmin, vmax=vmax))            

        # set colorbar
        if pt!=0:
            cbar = fig_tmean_space.colorbar(im,ax=ax,pad=0.002,orientation="horizontal")  
            cbar.ax.tick_params(labelsize='xx-small',pad=0.01, length=2)             
            cbar.set_label(label='($^\circ$C)', size='xx-small', rotation='horizontal', labelpad=-0.5)

        # set title
        if pt == 0:
            ax.set_title(title_str, fontsize='xx-small', fontweight='semibold')

        # set ylabel
        if j == 0:
            ax.set_ylabel(datetime.datetime.strftime(time_nldas[0],'%B %Y'), fontsize='xx-small', fontweight='semibold')

        # plot grids location
        if j == 0:
            m.scatter(lons, lats, marker = 'o', color='none', 
                      edgecolors='red', zorder=5, alpha=0.8)#magenta
                    
    #=====================================================
    # PART 3.7 plot Tmean in time
    print('plot Tmean in time')
    ax = fig_tmean_time.add_subplot(gs_tmean_time[pt,:])

    # extract point data
    nldas_point = tmean_nldas[:,xid,yid]            
    data_left = tmean_gmet_ens[:,xid,yid,:]
    data_right = tmean_stat_ens[:,xid,yid,:]  

    data_left = data_left.tolist()
    data_right = data_right.tolist()

    # boxplot left data
    positions=np.array(range(len(data_left)))*2.0-0.4
    color='blue'
    flierprops = dict(marker='+', markerfacecolor='none', markersize=2,
                  linestyle='none', markeredgecolor=color)
    bpl = ax.boxplot(data_left, positions=positions, widths=0.6, 
                     flierprops=flierprops, medianprops=dict(alpha=.8))
    set_box_color(bpl, color) # colors are from http://colorbrewer2.org/
    sc = ax.scatter(positions,nldas_point,s=4, c='k',alpha=1)

    # boxplot right data
    positions=np.array(range(len(data_left)))*2.0+0.4
    color='red'
    flierprops = dict(marker='+', markerfacecolor='none', markersize=2,
                  linestyle='none', markeredgecolor=color)
    bpr = ax.boxplot(data_right, positions=positions, widths=0.6, 
                     flierprops=flierprops, medianprops=dict(alpha=.8))
    set_box_color(bpr, color)
    sc = ax.scatter(positions,nldas_point,s=4, c='k',alpha=1)

    # Add a horizontal grid to the plot, but make it very light in color
    # so we can use it for reading data values but not be distracting
    ax.yaxis.grid(True, linestyle='-', which='major', color='lightgrey',alpha=0.5)
    ax.set_axisbelow(True)

    # xticks
    ticks = [datetime.datetime.strftime(t,'%d') for t in time_nldas]
    ax.set_xticklabels(ticks)
    ax.set_xticks(np.array(range(len(data_left)))*2.0)
    ax.tick_params(axis='both',labelsize='xx-small')

    # draw temporary red and blue lines and use them to create a legend
    ax.legend([sc, bpl["boxes"][0], bpr["boxes"][0]], ['NLADS', 'Spatial regression', 'Parametric'], 
              loc='upper right',fontsize='xx-small')

    ax.set_xlabel('Date', fontsize='xx-small') 
    ax.set_ylabel('Tmean ($^\circ$C)', fontsize='xx-small') 

    # set title
    title_str= '(%s) Tmean in %s'%(chr(ord('a')+pt), datetime.datetime.strftime(time_nldas[0],'%B %Y'))                         
    ax.set_title(title_str, fontsize='xx-small', fontweight='semibold')
    
#=====================================================           
# save
# --- save precip in space
output_filename = '%s_Precip_%s_xid%d_yid%d_spatial.png'%(test_folder, datetime.datetime.strftime(time_nldas[0],'%B_%Y'),xid, yid)
fig_pcp_space.savefig(os.path.join(output_dir, output_filename), dpi=dpi_value)
plt.close(fig_pcp_space)

# --- save precip in time
output_filename = '%s_Precip_%s_xid%d_yid%d_time.png'%(test_folder, datetime.datetime.strftime(time_nldas[0],'%B_%Y'),xid, yid)
fig_pcp_time.savefig(os.path.join(output_dir, output_filename), dpi=dpi_value)
plt.close(fig_pcp_time)

# --- save Tmean in space
output_filename = '%s_Tmean_%s_xid%d_yid%d_spatial.png'%(test_folder, datetime.datetime.strftime(time_nldas[0],'%B_%Y'),xid, yid)
fig_tmean_space.savefig(os.path.join(output_dir, output_filename), dpi=dpi_value)
plt.close(fig_tmean_space)

# --- save Tmean in time
output_filename = '%s_Tmean_%s_xid%d_yid%d_time.png'%(test_folder, datetime.datetime.strftime(time_nldas[0],'%B_%Y'),xid, yid)
fig_tmean_time.savefig(os.path.join(output_dir, output_filename), dpi=dpi_value)
plt.close(fig_tmean_time)

print('Done')

Create colormap
---- 2016-02-01 - 2016-02-29 ----


<ipython-input-3-05c92fb3bc2a>:80: RuntimeWarning: Mean of empty slice
  pcp_nldas_mean = np.nanmean(pcp_nldas, axis=0) #(y,x)
/glade/u/home/hongli/tools/miniconda3/envs/conda_hongli/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
<ipython-input-3-05c92fb3bc2a>:81: RuntimeWarning: Mean of empty slice
  pcp_stat_std_mean = np.nanmean(np.nanstd(pcp_stat_ens, axis=-1), axis=0)
<ipython-input-3-05c92fb3bc2a>:83: RuntimeWarning: Mean of empty slice
  pcp_gmet_std_mean = np.nanmean(pcp_gmet_std, axis=0)
<ipython-input-3-05c92fb3bc2a>:94: RuntimeWarning: Mean of empty slice
  tmean_nldas_mean = np.nanmean(tmean_nldas, axis=0) #(y,x)
<ipython-input-3-05c92fb3bc2a>:95: RuntimeWarning: Mean of empty slice
  tmean_stat_std_mean = np.nanmean(np.nanstd(tmean_stat_ens, axis=-1), axis=0)
<ipython-input-3-05c92fb3bc2a>:97: RuntimeWarning: Mean of empty slice
  tmean_gmet_std_mea

plot Precip in space
plot Precip in time
plot Tmean in space
plot Tmean in time
---- 2016-08-01 - 2016-08-31 ----


<ipython-input-3-05c92fb3bc2a>:80: RuntimeWarning: Mean of empty slice
  pcp_nldas_mean = np.nanmean(pcp_nldas, axis=0) #(y,x)
/glade/u/home/hongli/tools/miniconda3/envs/conda_hongli/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
<ipython-input-3-05c92fb3bc2a>:81: RuntimeWarning: Mean of empty slice
  pcp_stat_std_mean = np.nanmean(np.nanstd(pcp_stat_ens, axis=-1), axis=0)
<ipython-input-3-05c92fb3bc2a>:83: RuntimeWarning: Mean of empty slice
  pcp_gmet_std_mean = np.nanmean(pcp_gmet_std, axis=0)
<ipython-input-3-05c92fb3bc2a>:94: RuntimeWarning: Mean of empty slice
  tmean_nldas_mean = np.nanmean(tmean_nldas, axis=0) #(y,x)
<ipython-input-3-05c92fb3bc2a>:95: RuntimeWarning: Mean of empty slice
  tmean_stat_std_mean = np.nanmean(np.nanstd(tmean_stat_ens, axis=-1), axis=0)
<ipython-input-3-05c92fb3bc2a>:97: RuntimeWarning: Mean of empty slice
  tmean_gmet_std_mea

plot Precip in space
plot Precip in time
plot Tmean in space
plot Tmean in time
Done


In [4]:
data

array([[       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       ...,
       [0.30824798, 0.31744072, 0.31908718, ..., 0.15204985, 0.15317374,
        0.15459178],
       [0.322115  , 0.3239294 , 0.3264998 , ..., 0.14395365, 0.1490669 ,
        0.15212606],
       [0.32646263, 0.3263203 , 0.32391474, ..., 0.13311036, 0.13631953,
        0.14140235]], dtype=float32)